In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from timeit import default_timer as timer

from metrics import calc_rmse
from models.utils import df_to_rating_matrix
from models.matrix_factorization import MatrixFactorization, train_test_split_rating_mat,\
    MatrixFactorization_rating_matrix_version
from preprocessing.utils import optimize_dtypes, train_test_split_dataframe



print("Welcome to Haneul's Recommendation engine")
rating_df = pd.read_parquet("datasets/ml-25m/ratings_sampled.parquet").sample(10_000)
# FIXME : TypeError: No matching signature found
rating_df = optimize_dtypes(rating_df, verbose=False)
train_df, test_df = train_test_split_dataframe(rating_df, test_ratio=0.3, method='random',
                                               time_col=None, verbose=True)
user_col_nm, item_col_nm, rating_col_nm = "userId", "movieId", "rating"

emb_dim = 3
epoch = 10
lr = 0.01
_lambda = 0.01

n_users, n_items = train_df[[user_col_nm, item_col_nm]].nunique()
MF_model = MatrixFactorization(emb_dim, lr, _lambda, n_users, n_items)
MF_model.fit(train_df, user_col_nm, item_col_nm, rating_col_nm, epoch=epoch, verbose=True)

print("====== prediction on trained dataset ======")
train_df['pred_rating'] = MF_model.predict(train_df)
print(train_df.head())
print(f"RMSE = {round(calc_rmse(train_df['rating'], train_df['pred_rating']), 3)}")

Welcome to Haneul's Recommendation engine
train_df shape = (7000, 4)
test_df shape = (3000, 4)


C:\Users\haneu\Desktop\PROJECTS\Recommender-System\preprocessing\utils.py:36: RuntimeWarning: overflow encountered in half_scalars
  if diff*c < (float16_max - float16_min):


0 : RMSE =   3.6597
====== prediction on trained dataset ======
          userId  movieId  rating   timestamp  pred_rating
index                                                     
6255248    40530     6539     4.0  1358013059     0.153852
14504928   93910      474     3.0  1225290142     0.300144
10827570   70272   117176     4.5  1532158993     0.487915
15221027   98623     4214     4.0  1032160847     0.427911
4446224    29202      337     4.0  1107404091     0.316904
RMSE = 3.326


In [ ]:
recc_df = MF_model.batch_reccomendation(test_df)

In [ ]:
recc_df.head()

In [ ]:
userid = 115934
recc_df.loc[recc_df['userId'] == userid]

In [ ]:
recc_df.loc[recc_df['relevant_score'].isna()].head()

In [ ]:
recc_df.loc[~recc_df['relevant_score'].isna()].head()

In [ ]:
test_df['userId'].unique()

In [ ]:
# one by one
users = test_df['userId'].unique()
for userid in users:
    try:
        # warm-user
        user_emb = MF_model.userid2idx[userid]
        movieid = MF_model.get_topK(user_emb, topk=1)[MF_model.item_col]
        print(f'recommend user={userid} -> movieID = {movieid}')
        
    except KeyError:
        

In [ ]:
try: MF_model.userid2idx[userid]

In [2]:
MF_model.mp_df_sorted

mean  count
movieId             
3275      5.0      4
728       5.0      3
994       5.0      2
1057      5.0      2
1095      5.0      2
...       ...    ...
140121    0.5      1
153899    0.5      1
159441    0.5      1
190741    0.5      1
196511    0.5      1

[3023 rows x 2 columns]